### Data Extraction

In [ ]:
import gzip
import os
import json

def read_json(input_folder):
    all_records = [] 

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".json.gz"):
            input_path = os.path.join(input_folder, file_name)

            with gzip.open(input_path, 'rt', encoding='utf-8') as f:
                for line in f:
                    record = json.loads(line)
                    all_records.append(record)

    return all_records

input_folder = "./tunet/"

all_json_data = read_json(input_folder)

# Let's choose a subset to reduce the operation time later
all_json_data = all_json_data[:5000]


{
  "author": [
    {
      "fullName": "Ryan, Conor",
      "name": "Conor",
      "surname": "Ryan",
      "rank": 1,
      "pid": null
    },
    {
      "fullName": "Brazil, Jonathan",
      "name": "Jonathan",
      "surname": "Brazil",
      "rank": 2,
      "pid": null
    },
    {
      "fullName": "de Leastar, Eamonn",
      "name": "Eamonn",
      "surname": "Leastar",
      "rank": 3,
      "pid": null
    },
    {
      "fullName": "Cloney, James",
      "name": "James",
      "surname": "Cloney",
      "rank": 4,
      "pid": null
    }
  ],
  "openAccessColor": null,
  "publiclyFunded": true,
  "type": "publication",
  "language": {
    "code": "und",
    "label": "Undetermined"
  },
  "country": [
    {
      "code": "IE",
      "label": "Ireland",
      "provenance": {
        "provenance": "Inferred by OpenAIRE",
        "trust": "0.85"
      }
    }
  ],
  "subjects": [
    {
      "subject": {
        "scheme": "keyword",
        "value": "Walton Institute for Inform

In [31]:
!python3.11 -m pip install neo4j

### Data Conversion

In [62]:
import json
import csv

def flatten_main(record):
    indicators = record.get('indicators', {}) or {}
    citation_impact = indicators.get('citationImpact', {}) or {}
    return {
        'id': record.get('id', ''),
        'title': record.get('mainTitle', ''),
        'description': ' '.join(record.get('description', [])),
        'type': record.get('type', ''),
        'citationCount': citation_impact.get('citationCount', 0),
        'influence': citation_impact.get('influence', 0),
        'popularity': citation_impact.get('popularity', 0),
        'url': record.get('url', [None])[0],
    }

def flatten_language(record):
    languages = []
    languages.append({
        'id': record.get('id', ''),
        'language': record.get('language', {}).get('label', ''),
    })
    return languages

def flatten_publicationDate(record):
    date = []
    date.append({
        'id': record.get('id', ''),
        'publicationDate': record.get('publicationDate', ''),
    })
    return date

def flatten_publisher(record):
    publisher = []
    publisher.append({
        'id': record.get('id', ''),
        'publisher': record.get('publisher', ''),
    })
    return publisher

def flatten_authors(record):
    authors = []
    for author in record.get('author', []):
        authors.append({
            'id': record.get('id', ''),
            'fullName': author.get('fullName', ''),
            'rank': author.get('rank', '')
        })
    return authors

def flatten_keywords(record):
    keywords = []
    for subject in record.get('subjects', []):
        keywords.append({
            'id': record['id'],
            'keyword': subject['subject']['value']
        })
    return keywords

main_records = []
authors_records = []
keywords_records = []
languanges_records = []
publicationDate_records = []
publisher_records = []

for result in all_json_data:
    main_records.append(flatten_main(result))
    authors_records.extend(flatten_authors(result))
    keywords_records.extend(flatten_keywords(result))
    languanges_records.extend(flatten_language(result))
    publicationDate_records.extend(flatten_publicationDate(result))
    publisher_records.extend(flatten_publisher(result))

def write_csv(filename, fieldnames, records):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(records)

write_csv('main.csv', ['id', 'title', 'description', 'type', 'citationCount', 'influence', 'popularity', 'url'], main_records)
write_csv('authors.csv', ['id', 'fullName', 'rank'], authors_records)
write_csv('keywords.csv', ['id', 'keyword'], keywords_records)
write_csv('languages.csv', ['id', 'language'], languanges_records)
write_csv('publicationDate.csv', ['id', 'publicationDate'], publicationDate_records)
write_csv('publisher.csv', ['id', 'publisher'], publisher_records)


In [65]:
import pandas as pd

df = pd.read_csv("./main.csv")
df

,id,title,description,type,citationCount,influence,popularity,url
0,dedup_wf_002::09e1111bd38467e7a4737eefece389d3,A Rating Bureau Service for Next Generation IP...,The marked increase of IP-based services has r...,publication,0.0,2.983720e-09,3.489363e-10,NaN
1,dedup_wf_002::3b1d7daa1676df1f52431597574f53d8,The Silent Menace,"DAVID POWER SPEAKS TO DR. TERESA GRAHAM, CHAIR...",publication,0.0,2.983720e-09,5.948760e-10,NaN
2,dedup_wf_002::63f990d0b112ccb05deb68cabf66a047,Revisiting the Nature of Information Systems: ...,This paper suggests that the emergence of larg...,publication,0.0,2.983720e-09,3.696027e-10,NaN
3,dedup_wf_002::bb0398ec8d9d644a0492c738ceefb6e8,Identifying the factors that influence Irish h...,The purpose of this current study is to identi...,publication,0.0,2.983720e-09,4.949075e-10,NaN
4,dedup_wf_002::bcd5702e43a3545d63f2f7e9aa2664f6,Higher Education and Regional Economic Develop...,This presentation was delivered at a South Eas...,publication,0.0,2.983720e-09,8.197724e-10,NaN
...,...,...,...,...,...,...,...,...
4995,doi_dedup___::fda42090a31a9e8d1c88286d7be353f9,Channel Impulse Analysis of Light Propagation ...,Recent Brain-Machine Interfaces have shifted t...,publication,0.0,2.983720e-09,2.351643e-09,NaN
4996,doi_dedup___::fe11df355616cd298b81c45c455b5120,Volatility Spillovers Between Stock Returns an...,This paper investigates the nature of volatili...,publication,7.0,3.595735e-09,2.450056e-09,NaN
4997,doi_dedup___::fe87e4eb18e836d29b8bd0cfda77deb8,The Mares Conference on Marine Ecosystems Heal...,<jats:p>Marine environments are generally cons...,publication,0.0,2.983720e-09,1.183222e-09,NaN
4998,od______1248::09b32e37b5c87a047fb49abb3bc6bfa2,Reflections of a First Year Student from overs...,"Note from the Editors This short, reflective p...",publication,0.0,2.983720e-09,2.719802e-09,NaN


In [70]:
(df['type']).value_counts()

type
publication    3891
other           794
dataset         315
Name: count, dtype: int64

### Data Cleaning

In [ ]:
import numpy as np

# Remove empty publishers
df = pd.read_csv("./publisher.csv")
df['publisher'].replace('', np.nan).dropna()
df.to_csv("publisher.csv", index=False)

In [ ]:
df = pd.read_csv("./keywords.csv")
# To lowercase
df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
# Then remove duplicates
df = df.drop_duplicates()
df.to_csv("keywords.csv", index=False)

/var/folders/bw/rmt54fn96c5cgqt13_6_0zzh0000gn/T/ipykernel_2032/4223540635.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


### Relationship Creation and Data Upload

In [74]:
from neo4j import GraphDatabase
import csv

uri = "neo4j+s://6bf30dc9.databases.neo4j.io" 
username = "neo4j"
password = "EZwPS1wiwdWkxc1vwZNEDRoDVpi37G2TDF4i45-Oq5c"
driver = GraphDatabase.driver(uri, auth=(username, password))

def upload_data_from_csv(file_path, query):
    with driver.session() as session:
        with open(file_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            for row in reader:
                session.run(query, row)

query_main = """
MERGE (r:Record {id: $id})
SET r.title = $title, 
    r.description = $description, 
    r.type = $type, 
    r.citationCount = $citationCount, 
    r.influence = $influence, 
    r.popularity = $popularity,
    r.url = $url
"""

query_authors = """
MERGE (a:Author {fullName: $fullName})
SET a.rank = $rank
WITH a
MATCH (r:Record {id: $id})
MERGE (r)-[:HAS_AUTHOR]->(a)
"""

query_keywords = """
MERGE (k:Keyword {keyword: $keyword})
WITH k
MATCH (r:Record {id: $id})
MERGE (r)-[:HAS_KEYWORD]->(k)
"""

query_publisher = """
MERGE (k:Publisher {publisher: $publisher})
WITH k
MATCH (r:Record {id: $id})
MERGE (r)-[:HAS_PUBLISHER]->(k)
"""

query_publicationDate = """
MERGE (k:PublicationDate {publicationDate: $publicationDate})
WITH k
MATCH (r:Record {id: $id})
MERGE (r)-[:HAS_PUBLICATION_DATE]->(k)
"""

query_language = """
MERGE (k:Language {language: $language})
WITH k
MATCH (r:Record {id: $id})
MERGE (r)-[:HAS_LANGUAGE]->(k)
"""

upload_data_from_csv('main.csv', query_main)
upload_data_from_csv('authors.csv', query_authors)
upload_data_from_csv('keywords.csv', query_keywords)
upload_data_from_csv('publisher.csv', query_publisher)
upload_data_from_csv('publicationDate.csv', query_publicationDate)
upload_data_from_csv('languages.csv', query_language)

driver.close()
